In [ ]:
#Numeric fucnitons

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate()

data = [
    (1, -10.5),
    (2, 20.3),
    (3, 30.7),
    (4, -5.2)
]

df = spark.createDataFrame(data, ["id", "value"])
df.show()

+---+-----+
| id|value|
+---+-----+
|  1|-10.5|
|  2| 20.3|
|  3| 30.7|
|  4| -5.2|
+---+-----+



In [2]:
# Value	round	bround
# 2.5	  3	    2


# df.select(
#     abs("value").alias("abs_value"),
#     round("value", 0).alias("round_val"),
#     bround("value", 0).alias("round_val"),
#     # Smallest integer ≥ value.
#     ceil(col("value")).alias("ceil"),
#     floor("value").alias("floor_ans"),
#     sqrt("value"),
#     pow("value", 2),
#     exp("value"),
#     # log(2, "value"),
#     # Base 10 log.
#     log10("value"),
#     # Random number between 0–1.
#     rand().alias("random"),
#     # Random normal distribution.
#     randn().alias("random_nn"),
#     greatest("value", lit(10))


# ).show()

df.select(
    abs("value").alias("abs_value"),
    round("value", 0).alias("round_val"),
    bround("value", 0).alias("bround_val"),
    ceil("value").alias("ceil"),
    floor("value").alias("floor"),
    sqrt(abs("value")).alias("sqrt_fixed"),
    pow("value", 2).alias("power"),
    exp("value").alias("exp"),
    log(2.0, abs("value")).alias("log_base2"),
    log10(abs("value")).alias("log10"),
    rand().alias("rand"),
    randn().alias("randn"),
    greatest("value", lit(20)).alias("greatest_row"), # val vs lit(val) greater used
    least("value" , lit(20)).alias("least_amg_col") ,  # val vs lit(val) least used
).show()



+---------+---------+----------+----+-----+-----------------+------------------+--------------------+------------------+------------------+-------------------+-------------------+------------+-------------+
|abs_value|round_val|bround_val|ceil|floor|       sqrt_fixed|             power|                 exp|         log_base2|             log10|               rand|              randn|greatest_row|least_amg_col|
+---------+---------+----------+----+-----+-----------------+------------------+--------------------+------------------+------------------+-------------------+-------------------+------------+-------------+
|     10.5|    -11.0|     -10.0| -10|  -11| 3.24037034920393|            110.25|2.753644934974715...|3.3923174227787602| 1.021189299069938| 0.6843363747599158| 1.9549071471914765|        20.0|        -10.5|
|     20.3|     20.0|      20.0|  21|   20|4.505552130427524|412.09000000000003| 6.549045121532385E8| 4.343407822297814| 1.307496037913213|0.18688954043208272|  2.526863618

In [19]:
df.select(
    (col("value") + 10).alias("plus"),
    (col("value") * 2).alias("double"),
    (col("value") / 2).alias("half")
).show()

+----+------+-----+
|plus|double| half|
+----+------+-----+
|-0.5| -21.0|-5.25|
|30.3|  40.6|10.15|
|40.7|  61.4|15.35|
| 4.8| -10.4| -2.6|
+----+------+-----+



In [51]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.functions import countDistinct

spark = SparkSession.builder.appName("AggExample").getOrCreate()

data = [
    ("IT", 50000),
    ("IT", 60000),
    ("HR", 40000),
    ("HR", 45000),
    ("Sales", 30000)
]

columns = ["department", "salary"]

df = spark.createDataFrame(data, columns)
df.show()

+----------+------+
|department|salary|
+----------+------+
|        IT| 50000|
|        IT| 60000|
|        HR| 40000|
|        HR| 45000|
|     Sales| 30000|
+----------+------+



In [27]:
df.select(
    sum("salary").alias("total_salary"),
    avg("salary").alias("average_salary"),
    max("salary").alias("max_salary"),
    min("salary").alias("min_salary"),
    count("salary").alias("count_salary")
).show()

+------------+--------------+----------+----------+------------+
|total_salary|average_salary|max_salary|min_salary|count_salary|
+------------+--------------+----------+----------+------------+
|      225000|       45000.0|     60000|     30000|           5|
+------------+--------------+----------+----------+------------+



In [29]:
# Using .agg() Separately (No groupBy)

# Same thing

df.agg(
    sum("salary").alias("total_salary"),
    avg("salary").alias("average_salary"),
    max("salary").alias("max_salary"),
    min("salary").alias("min_salary")
).show()

+------------+--------------+----------+----------+
|total_salary|average_salary|max_salary|min_salary|
+------------+--------------+----------+----------+
|      225000|       45000.0|     60000|     30000|
+------------+--------------+----------+----------+



In [53]:
# Using groupBy + agg

# Now aggregation per department.

df.groupBy("department").agg(
    sum("salary").alias("total_salary"),
    avg("salary").alias("average_salary"),
    max("salary").alias("max_salary"),
    min("salary").alias("min_salary"),
    count("salary").alias("employee_count")
).show()


+----------+------------+--------------+----------+----------+--------------+
|department|total_salary|average_salary|max_salary|min_salary|employee_count|
+----------+------------+--------------+----------+----------+--------------+
|        IT|      110000|       55000.0|     60000|     50000|             2|
|     Sales|       30000|       30000.0|     30000|     30000|             1|
|        HR|       85000|       42500.0|     45000|     40000|             2|
+----------+------------+--------------+----------+----------+--------------+



In [52]:
df.agg(
    countDistinct("department").alias("unique_departments")
).show()

+------------------+
|unique_departments|
+------------------+
|                 3|
+------------------+



In [43]:
# first() (Needs groupBy)
df.groupBy("department").agg(
    first("salary").alias("first_salary")
).show()
# Returns first salary value in each department group

+----------+------------+
|department|first_salary|
+----------+------------+
|        IT|       50000|
|     Sales|       30000|
|        HR|       40000|
+----------+------------+



In [44]:
# last() (Needs groupBy)
df.groupBy("department").agg(
    last("salary").alias("last_salary")
).show()
# Returns last salary value in each department group

+----------+-----------+
|department|last_salary|
+----------+-----------+
|        IT|      60000|
|     Sales|      30000|
|        HR|      45000|
+----------+-----------+



In [46]:
# collect_list() (Needs groupBy)
df.groupBy("department").agg(
    collect_list("salary").alias("salary_list")
).show()
# Collects salaries into list (keeps duplicates)

+----------+--------------+
|department|   salary_list|
+----------+--------------+
|        IT|[50000, 60000]|
|     Sales|       [30000]|
|        HR|[40000, 45000]|
+----------+--------------+



In [47]:
# collect_set() (Needs groupBy)
df.groupBy("department").agg(
    collect_set("salary").alias("unique_salary_set")
).show()
# Collects unique salaries into set (removes duplicates)

+----------+-----------------+
|department|unique_salary_set|
+----------+-----------------+
|        IT|   [50000, 60000]|
|     Sales|          [30000]|
|        HR|   [40000, 45000]|
+----------+-----------------+



In [48]:
# skewness()
df.agg(
    skewness("salary").alias("salary_skewness")
).show()
# Measures asymmetry of salary distribution

+--------------------+
|     salary_skewness|
+--------------------+
|-2.44140625000000...|
+--------------------+



In [49]:
df.rollup("department").agg(
    sum("salary").alias("total_salary"),
    grouping("department").alias("grouping_flag")
).show()
# grouping() returns 1 if column is aggregated (NULL level), else 0

+----------+------------+-------------+
|department|total_salary|grouping_flag|
+----------+------------+-------------+
|      NULL|      225000|            1|
|        IT|      110000|            0|
|        HR|       85000|            0|
|     Sales|       30000|            0|
+----------+------------+-------------+



In [35]:
df.agg(
    approx_count_distinct("department").alias("approx_unique_departments")
).show()
# Faster approximate unique count (uses HyperLogLog algorithm)

+-------------------------+
|approx_unique_departments|
+-------------------------+
|                        3|
+-------------------------+



In [ ]:
# stddev()
df.agg(
    stddev("salary").alias("stddev_salary")
).show()
# Calculates sample standard deviation of salary

In [40]:
# stddev_samp()
df.agg(
    stddev_samp("salary").alias("sample_stddev_salary")
).show()
# Explicit sample standard deviation

+--------------------+
|sample_stddev_salary|
+--------------------+
|  11180.339887498947|
+--------------------+



In [41]:
# variance()
df.agg(
    variance("salary").alias("variance_salary")
).show()
# Sample variance

+--------------------+
|     variance_salary|
+--------------------+
|1.2499999999999997E8|
+--------------------+



In [42]:
# var_samp()
df.agg(
    var_samp("salary").alias("sample_variance_salary")
).show()
# Sample variance explicitly

+----------------------+
|sample_variance_salary|
+----------------------+
|  1.2499999999999997E8|
+----------------------+

